In [1]:
#calling API function

import flickrapi
import json

api_key = u'f950122b83b682c546201f10d33edffe'
api_secret = u'057c65cd7fe1b2c8'

#flickr = flickrapi.FlickrAPI(api_key, api_secret)
#for json format
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

In [ ]:
#earlier version

extras='geo,description,tags'
MacArthurPark = flickr.photos.search(tags='MacArthur Park', bbox = '-118.898278,33.704902,-118.161021,34.32848',
                                     method_name='flickr', per_page=500, extras=extras,format='parsed-json')
photos = MacArthurPark['photos']
from pprint import pprint
pprint(photos)

In [58]:
import pandas as pd

def get_parks(num_pages):
    park_list = []
    for i in range(1, num_pages+1): #range documentation starts at 0, +1 ensures we pull the page number we feed the function below
        extras = 'geo,description,tags'
        tags = ['MacArthur Park, Woodley Avenue Park, Rio de Los Angeles State Park, Runyon Canyon, Temescal Gateway, Heidelberg Park, Hancock Park, Franklin Canyon Park, Angels Gate, Coldwater Canyon, Chatsworth Park South, Cheviot Hills, O''Melveny Park']
        parks_LA = flickr.photos.search(tags=tags, bbox = '-118.898278,33.704902,-118.161021,34.32848',
                                        method_name='flickr', page=i, per_page=500, extras=extras)  
        
        #.extend combines each page of search results together: https://www.programiz.com/python-programming/methods/list/extend
        park_list.extend(parks_LA['photos']['photo']) #pulls data from each individual photo 
        
    #reorients the data and converts to pandas df: https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe
    df = pd.DataFrame.from_dict(park_list, orient='columns') 
    
    return df

parks_data = get_parks(12) #pulls data from all 12 pages of photos

In [59]:
print(parks_data['tags'])

0       koreanfriendshipbell korean friendshipbell fri...
1         englishhome architecture losangeles hancockpark
2       angelsgate losangelesharbor sanpedro sunrise c...
3       sanpedro losangeles harbor california southern...
4       jrglongbeach smoke water harbor hiking sanpedr...
                              ...                        
2709    dog topv111 losangeles vizsla ridgeback runyon...
2710      dog losangeles runyoncanyon runyon weimardoodle
2711    dog losangeles vizsla buddy ridgeback runyonca...
2712          losangeles runyoncanyon runyon weimardoodle
2713      dog losangeles runyoncanyon runyon weimardoodle
Name: tags, Length: 2714, dtype: object


In [60]:
parks_tags = parks_data['tags'].str.lower()
type(parks_tags)

pandas.core.series.Series

In [61]:
tags_list = parks_data["tags"].tolist()
print(tags_list)
type(tags_list)

['koreanfriendshipbell korean friendshipbell friendship bell sanpedro san pedro california ca angelsgatepark angelsgate park angels gate coastalsanpedro', 'englishhome architecture losangeles hancockpark', 'angelsgate losangelesharbor sanpedro sunrise colorful sky cloudscapes', 'sanpedro losangeles harbor california southerncalifornia angelsgate', 'jrglongbeach smoke water harbor hiking sanpedro angelsgate glint', 'hancockpark losangeles bw blackandwhite d750 monochrome museum nikon park pole silverefexpro tar labreatarpits handprints', 'bw barrier blackandwhite d750 monochrome nikon park silverefexpro sloths statues trees labreatarpits hancockpark losangeles', 'stoneypointpark santasusanapass santasusanapassrd chatsworthca sanfernandovalley la losangeles socal california simihills chatsworthparksouth mountains hills rocks boulders topangacanyonblvd', 'losangeles ca california subway metro public tatyana transit mass red tile escalator exit entrance stairs reflection macarthurpark west

list

In [62]:
tags_str = ''.join(tags_list)
tags_str

'koreanfriendshipbell korean friendshipbell friendship bell sanpedro san pedro california ca angelsgatepark angelsgate park angels gate coastalsanpedroenglishhome architecture losangeles hancockparkangelsgate losangelesharbor sanpedro sunrise colorful sky cloudscapessanpedro losangeles harbor california southerncalifornia angelsgatejrglongbeach smoke water harbor hiking sanpedro angelsgate glinthancockpark losangeles bw blackandwhite d750 monochrome museum nikon park pole silverefexpro tar labreatarpits handprintsbw barrier blackandwhite d750 monochrome nikon park silverefexpro sloths statues trees labreatarpits hancockpark losangelesstoneypointpark santasusanapass santasusanapassrd chatsworthca sanfernandovalley la losangeles socal california simihills chatsworthparksouth mountains hills rocks boulders topangacanyonblvdlosangeles ca california subway metro public tatyana transit mass red tile escalator exit entrance stairs reflection macarthurpark westlake someoneleftmycakeoutintherai

In [53]:
#tokenize and lemmatize code found: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jacquelineadams/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacquelineadams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacquelineadams/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
import pyLDAvis
import pyLDAvis.gensim_models
import re 
import gensim

/Users/jacquelineadams/opt/anaconda3/envs/urban_datascience/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [55]:
swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]

wordlists = [[word for word in word_tokenize(tags_str.lower()) if word not in swords] for tag in tags_str]

KeyboardInterrupt: 

In [ ]:
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = 0.9, eta=0.9)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

In [ ]:
remove_words = ['macarthurpark', 'ca', 'losangeles', 'park', 'california', 'southerncalifornia', 'macarthurparkla',
                'usa', 'socal', 'lac', 'la', 'dtla', 'downtownlosangeleslosangeles']

In [ ]:
wordlists = [[word for word in word_tokenize(tags_str.lower()) if word not in remove_words] for tag in tags_str]
#wordlists = [wl for wl in wordlists if 'mcarthurpark' not in wl]

dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = 0.9, eta=0.9)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

In [ ]:
#ARCHIVE

import requests
from bs4 import BeautifulSoup

url= 'https://www.flickr.com/search/?tags=MacArthur%20Park'
r = requests.get(url)
soup = BeautifulSoup(r.content)
print(soup.prettify())

In [ ]:
extras='url_sq,url_t,url_s,url_q,url_m,url_n,url_z,url_c,url_l,url_o'
cats = flickr.photos.search(text='kitten', per_page=5, extras=extras)
photos = cats['photos']
from pprint import pprint
pprint(photos)

In [ ]:
#pull info from first photo in the set
item1 = photos['photo'][0]  
print(item1)

In [ ]:
#convert to a dataframe
import pandas as pd
df_parks = pd.DataFrame(photos['photo'])
df_parks.shape[0]